Я работаю в добывающей компании «ГлавРосГосНефть». Руководству компании, с моей помощью нужно решить, где бурить новую скважину.

Мне предоставили пробы нефти в трёх регионах, в каждом находится по 10 000 месторождений. В каждой пробе измерили качество нефти и объём её запасов. Нужно построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Необходимо проанализировать возможную прибыль и риски техникой Bootstrap.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании 
  и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.
Описание данных:

Данные геологоразведки трёх регионов находятся в файлах:

/datasets/geo_data_0.csv
/datasets/geo_data_1.csv
/datasets/geo_data_2.csv
Для упрощения:

- будем называть регионы месторождениями нефти
- порядковый номер месторождений сохраним так же как и названия файлов трех регионов: 0, 1, 2.
Признаки:

id — уникальный идентификатор скважины;
f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
product — объём запасов в скважине (тыс. баррелей).

1  Шаг. Загрузка и подготовка данных

Устанавливаем новые и обновляем уже имеющиеся библиотеки в целях их совместной корректной работы с визуализацией и при взаимодействии с моделями. Не отображаем процесс инсталляции для экономии места.

Импортируем библиотеки

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn
import os
import phik
import warnings
from phik.report import plot_correlation_matrix
from phik import report                        
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error

Считаем файлы данных и сохраним их в датафреймы. Поиск пути к файлам работает как для "Юпитера", как и для локального положения.

In [ ]:
pth1j = '/datasets/geo_data_0.csv'
pth1 = 'geo_data_0.csv'
pth2j = '/datasets/geo_data_1.csv'
pth2 = 'geo_data_1.csv'
pth3j = '/datasets/geo_data_2.csv'
pth3 = 'geo_data_2.csv'

if os.path.exists(pth1j):
    data0 = pd.read_csv(pth1j)
elif os.path.exists(pth1):
    data0 = pd.read_csv(pth1)
else:
    print('Something go wrong with path №1')
    
if os.path.exists(pth2j):
    data1 = pd.read_csv(pth2j)
elif os.path.exists(pth2):
    data1 = pd.read_csv(pth2)
else:
    print('Something go wrong with path №2')
    
if os.path.exists(pth3j):
    data2 = pd.read_csv(pth3j)
elif os.path.exists(pth3):
    data2 = pd.read_csv(pth3)
else:
    print('Something go wrong with path №3')

Для удобства установим внутренний индекс как колонку id для каждого датафрейма.

In [ ]:
data0 = data0.set_index('id')
data1 = data1.set_index('id')
data2 = data2.set_index('id')

Проверим, что данные в датафреймах соответствуют описанию:

In [ ]:
print('')
print('\033[1m','Данные геологоразведки первого региона')
data0.head()

In [ ]:
data0.info()

In [ ]:
data0.duplicated().sum()
np.int64(0)
print('')
print('\033[1m','Данные геологоразведки второго региона')
data1.head()

In [ ]:
data1.duplicated().sum()
np.int64(0)
print('')
print('\033[1m','Данные геологоразведки третьего региона')

In [ ]:
data2.head()

In [ ]:
data2.info()

In [ ]:
data2.duplicated().sum()
np.int64(0)

Напишем функцию для попарного вывода гистограммы и боксплота каждого из признаков в наших датафреймах:

In [ ]:
def plotandbox(data, feature, hist_color='darkslategray'):
    fig, axs = plt.subplots(nrows = 1 , ncols = 2, figsize = (12, 4))
    sn.histplot(data = data, x = feature, ax=axs[0], bins = 50, kde=True, color=hist_color, edgecolor='black')
    axs[0].set_xlabel(feature)
    axs[0].set_ylabel('частота')
    sn.boxplot(data = data, x = feature, ax=axs[1], color='azure')
    axs[1].set_xlabel(feature)
    fig.suptitle(f'Анализ распределения {feature}')
    plt.show()

Графический анализ данных

In [ ]:
data0.describe()

In [ ]:
plotandbox(data0, 'f0')
plotandbox(data0, 'f1', 'plum')
plotandbox(data0, 'f2', 'sandybrown')
plotandbox(data0, 'product', 'cadetblue')

Вывод:
- у признака f2 замечены "усы"
- думаю что усы можно не обрезать, что бы сохранить количественное единообразие данных
data1.describe()

In [ ]:
plotandbox(data1, 'f0')
plotandbox(data1, 'f1', 'plum')
plotandbox(data1, 'f2', 'sandybrown')
plotandbox(data1, 'product', 'cadetblue')

Вывод:
- у признака f1 замечены "усы"
- странный, периодический характер данных гистограмм f2 и product - это не выбросы
- замечено сильное внешнее сходство гистограмм признака f2 и целевого признака product
- все значения признаков f2 и product выстроились по шести кластерам, встречаемость их выше почти в 3 раза

In [ ]:
data2.describe()

In [ ]:
plotandbox(data2, 'f0')
plotandbox(data2, 'f1', 'plum')
plotandbox(data2, 'f2', 'sandybrown')
plotandbox(data2, 'product', 'cadetblue')

Вывод:
- в data2 у всех признаков кроме целевого замечены "усы"
- думаю что усы можно не обрезать, что бы сохранить количественное единообразие данных

Корреляционный анализ данных

Напишем функцию для вывода корреляций наших датафреймов:

In [ ]:
def corephima(dataframe_name, interval_cols=['f0', 'f1', 'f2', 'product'], title=r"корреляция", fontsize_factor=1.5, figsize=(6, 4)):
    phik_overview = dataframe_name.phik_matrix(interval_cols)
    plot_correlation_matrix(
        phik_overview.values,
        x_labels=phik_overview.columns,
        y_labels=phik_overview.index,
        title=title,
        fontsize_factor=fontsize_factor,
        figsize=figsize
    )

In [ ]:
corephima(data0)

Выводы:
- Сильная прямая корреляция у f1 и f0.
- Негативных моментов потенциально влияющих на модель - не замечено.

In [ ]:
corephima(data1)

Самым значимым признаком в data1 является f2:

- вероятнее всего, он будет играть основную роль в предсказаниях
- модель может стать менее робастной, если признак содержит шум или выбросы
- считаем что мультиколлинеарности нет, так как нет других сильных связей с f0 и f1

In [ ]:
corephima(data2)

- Негативных моментов потенциально влияющих на модель исходя из анализа корреляций - не замечено.

Выводы по ШАГУ-1:
- импортированы необходимые библиотеки
- колонку id установили в качестве внутреннего индекса, для удобства
- ознакомились с датасетами: тип данных - везде вещественный, тип переменной - везде количественный
- датасеты проверены на пропуски и дубликаты, которых обнаружено не было.

2  Шаг. Обучение и проверка модели

In [ ]:
warnings.filterwarnings("ignore")

RANDOM_STATE = 42
X0 = data0.drop('product', axis = 1)
y0 = data0['product']
X0_train, X0_valid, y0_train, y0_valid = train_test_split(X0, y0, test_size=0.25, random_state=RANDOM_STATE) 
scaler0 = StandardScaler()
X0_train_scaler = scaler0.fit_transform(X0_train)
X0_valid_scaler = scaler0.transform(X0_valid)
model0 = LinearRegression()
model0.fit(X0_train_scaler, y0_train)
predict0 = model0.predict(X0_valid_scaler)
mean0 = predict0.mean()
rmse0 = mean_squared_error(y0_valid, predict0, squared=False)
print(f'Средний запас предсказанного сырья: {mean0}, значение метрики RMSE: {rmse0}')

data1:

In [ ]:
X1 = data1.drop('product', axis = 1)
y1 = data1['product']
X1_train, X1_valid, y1_train, y1_valid = train_test_split(
        X1, 
        y1, 
        test_size=0.25,
        random_state=RANDOM_STATE
) 
scaler1 = StandardScaler()
X1_train_scaler = scaler1.fit_transform(X1_train)
X1_valid_scaler = scaler1.transform(X1_valid)
model1 = LinearRegression()
model1.fit(X1_train_scaler, y1_train)
predict1 = model1.predict(X1_valid_scaler)
mean1 = predict1.mean()
rmse1 = mean_squared_error(y1_valid, predict1, squared=False)
print(f'Средний запас предсказанного сырья: {mean1}, значение метрики RMSE: {rmse1}')

data2:

In [ ]:
X2 = data2.drop('product', axis = 1)
y2 = data2['product']
X2_train, X2_valid, y2_train, y2_valid = train_test_split(
        X2, 
        y2, 
        test_size=0.25,
        random_state=RANDOM_STATE
) 
scaler2 = StandardScaler()
X2_train_scaler = scaler2.fit_transform(X2_train)
X2_valid_scaler = scaler2.transform(X2_valid)
model2 = LinearRegression()
model2.fit(X2_train_scaler, y2_train)
predict2 = model2.predict(X2_valid_scaler)
mean2 = predict2.mean()
rmse2 = mean_squared_error(y2_valid, predict2, squared=False)
print(f'Средний запас предсказанного сырья: {mean2}, значение метрики RMSE: {rmse2}')

Выводы:
- наибольший средний запас предсказанного сырья у второго месторождения (94.8) 
- на втором месте с незначительной разницей идёт нулевое месторождение (92.4)
- наименьшие запасы у первого месторождения (68.7)
- модель первого месторождения работает точнее всего (значение метрики RMSE там меньше единицы, а чем ближе 
  значение метрики к нулю, тем точнее модель. В тоже время у других месторождений значение метрики в районе 40)

3  Шаг. Подготовка к расчёту прибыли

Для разработки выбирают 200 лучших скважин:

In [ ]:
n = 200

Бюджет на разработку скважин 10 млрд. рублей:

In [ ]:
budget = 10 * 10**9

Доход с единицы продукта:

In [ ]:
revenue = 450000

Сколько выделяется денег на одну скважину:

In [ ]:
budget_one = budget/n

Достаточный объем сырья, для безубыточной разработки новой скважины:

In [ ]:
volume = budget_one / revenue

Вывод:
- в каждой скважине объем сырья должен быть не меньше, чем 111.111, что бы не возникло убытков
- если объем будет меньше, то затраты на бурение не окупятся

In [ ]:
delta0 = mean0 - volume
delta1 = mean1 - volume
delta2 = mean2 - volume
display(delta0,delta1,delta2)
np.float64(-18.712311204533435)
np.float64(-42.39823307197351)
np.float64(-16.340087233451726)

Вывод:

- средние запасы в каждом регионе меньше, необходимого для безубыточности объема
- исходя из графиков, найти 200 месторождений с объемом продукта выше чем точка безубыточности будет не сложно.

Нужно найти 200 лучших месторождений с как можно большими залежами нефти, что бы максимизировать прибыль. 

4  Шаг. Расчёт рисков и прибыли для каждого региона

In [ ]:
def profit(data, y_true, y_predict):
    data['predict'] = y_predict
    data['true'] = y_true
    data = data.sort_values(by = 'predict', ascending = False)
    data = data.head(200)
    profite = data['true'].sum()*450000 - budget
    return profite

In [ ]:
state = np.random.RandomState(12345)

Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли:

In [ ]:
prib0 = []
for i in range(1000):
    subsample = X0_valid.sample(n=500, replace = True, random_state=state)
    prib0.append(profit(subsample, y0_valid, pd.DataFrame(predict0, index = X0_valid.index)))
    
prib1 = []
for i in range(1000):
    subsample = X1_valid.sample(n=500, replace = True, random_state=state)
    prib1.append(profit(subsample, y1_valid, pd.DataFrame(predict1, index = X1_valid.index)))
    
prib2 = []
for i in range(1000):
    subsample = X2_valid.sample(n=500, replace = True, random_state=state)
    prib2.append(profit(subsample, y2_valid, pd.DataFrame(predict2, index = X2_valid.index)))

Средняя прибыль нулевого региона:

In [ ]:
pd.Series(prib0).mean()
np.float64(406278783.42441905)

Найдём 95% доверительный интервал для нулевого региона:

In [ ]:
lower = pd.Series(prib0).quantile(0.025)
upper = pd.Series(prib0).quantile(0.975)
print(f'95%-доверительный интервал для нулевого региона от {lower} до {upper}')

Вычислим риск убытков в нулевом регионе:

In [ ]:
len([i for i in prib0 if i<0])/len(prib0)*100 # Процент отрицательных значений в нулевом регионе

Средняя прибыль первого региона:

In [ ]:
pd.Series(prib1).mean()
np.float64(441504277.5922549)

Найдём 95% доверительный интервал для первого региона:

In [ ]:
lower = pd.Series(prib1).quantile(0.025)
upper = pd.Series(prib1).quantile(0.975)
print(f'95%-доверительный интервал для первого региона от {lower} до {upper}')

Вычислим риск убытков в первом регионе:

In [ ]:
len([i for i in prib1 if i<0])/len(prib1)*100 # Процент отрицательных значений в первом регионе
1.6

Средняя прибыль второго региона:

In [ ]:
pd.Series(prib2).mean()
np.float64(385213195.91415244)

Найдём 95% доверительный интервал для второго региона:

In [ ]:
lower = pd.Series(prib2).quantile(0.025)
upper = pd.Series(prib2).quantile(0.975)
print(f'95%-доверительный интервал для второго региона от {lower} до {upper}')

Вычислим риск убытков во втором регионе:

In [ ]:
len([i for i in prib2 if i<0])/len(prib2)*100 # Процент отрицательных значений во втором регионе
7.8

Итоговый вывод:

Для разработки месторождений предлагается регион №1 (второй по счёту).

Обоснование:

- Это единственный регион с вероятностью убытков меньше 2.5 
- В доверительном интервале нет отрицательных значений, значит 95% значений - прибыль, а не убытки. 
- Среднее значение прибыли в этом регионе - 441504277.5922549 
Поэтому именно этот регион лучше всего подойдёт для разработки